# Counting words

Because what's a parallel computing demo without counting words?

Some utilitiles for excluding commmon phrases and normalizing words

In [3]:
import re
non_word = re.compile(r'[\W\d]+', re.UNICODE)

def normalize_word(word):
    """normalize a word
    
    simply strips non-word characters and case
    """
    word = word.lower()
    word = non_word.sub('', word)
    return word

common_words = {
'the','of','and','in','to','a','is','it','that','which','as','on','by',
'be','this','with','are','from','will','at','you','not','for','no','have',
'i','or','if','his','its','they','but','their','one','all','he','when',
'than','so','these','them','may','see','other','was','has','an','there',
'more','we','footnote', 'who', 'had', 'been',  'she', 'do', 'what',
'her', 'him', 'my', 'me', 'would', 'could', 'said', 'am', 'were', 'very',
'your', 'did', 'not',
}

def yield_words(filename):
    """returns a generator of words in a file"""
    import io
    with io.open(filename, errors='replace') as f:
        for line in f:
            for word in line.split():
                word = normalize_word(word)
                if word:
                    yield word


A function that reads a file, and returns a dictionary
with string keys of phrases of `n` words,
whose values

In [4]:
def ngrams(filename, n=1):
    """compute ngram counts for the contents of a file"""
    word_iterator = yield_words(filename)
    counts = {}
    def _count_gram(gram):
        common = sum(word in common_words for word in gram)
        if common > n / 2.0:
            # don't count ngrams that are >= 50% common words
            return
        sgram = ' '.join(gram)
        counts.setdefault(sgram, 0)
        counts[sgram] += 1
    
    gram = []
    
    # get the first word
    while len(gram) < n:
        try:
            word = next(word_iterator)
            if not word:
                continue
        except StopIteration:
            return counts
        else:
            gram.append(word)
    
    _count_gram(gram)

    while True:
        try:
            word = next(word_iterator)
        except StopIteration:
            break
        else:
            gram.append(word)
            gram.pop(0)
            _count_gram(gram)
    return counts
            

In [5]:
%%writefile cathat.txt
the cat in the hat is a cat whose hat is big.

Overwriting cathat.txt


In [6]:
ngrams('cathat.txt', 1)

{'big': 1, 'cat': 2, 'hat': 2, 'whose': 1}

In [7]:
ngrams('cathat.txt', 2)

{'a cat': 1,
 'cat in': 1,
 'cat whose': 1,
 'hat is': 2,
 'is big': 1,
 'the cat': 1,
 'the hat': 1,
 'whose hat': 1}

Now fetch some interesting data from Project Gutenberg:

In [8]:
try: 
    from urllib.request import urlretrieve # py3
except ImportError:
    from urllib import urlretrieve # py2

davinci_url = "http://www.gutenberg.org/files/5000/5000-8.txt"

if not os.path.exists('davinci.txt'):
    # download from project gutenberg
    print("Downloading Da Vinci's notebooks from Project Gutenberg")
    urlretrieve(davinci_url, 'davinci.txt')

In [9]:
import sys

def print_common(freqs, n=10):
    """Print the n most common keys by count."""
    words, counts = freqs.keys(), freqs.values()
    items = zip(counts, words)
    items = sorted(items, reverse=True)
    justify = 0
    for (count, word) in items[:n]:
        justify = max(justify, len(word))
    
    for (count, word) in items[:n]:
        print(word.rjust(justify), count)


In [10]:
# Run the serial version
print("Serial word frequency count:")
%time counts = ngrams('davinci.txt', 1)
print_common(counts, 10)


Serial word frequency count:
CPU times: user 632 ms, sys: 6.38 ms, total: 638 ms
Wall time: 638 ms
   light 849
     eye 589
    same 536
  shadow 507
    body 454
 between 445
   water 425
leonardo 416
    seen 415
    into 403


Let's split the file

In [11]:
# split the davinci.txt into one file per engine:
text = open('davinci.txt', encoding='latin1', errors='replace').read()
lines = text.splitlines()
nlines = len(lines)
n = 10

block = nlines//n
for i in range(n):
    chunk = lines[i*block:(i+1)*(block)]
    with open('davinci%i.txt' % i, 'w') as f:
        f.write('\n'.join(chunk))

In [12]:
import os
cwd = os.path.abspath(os.getcwd())
fnames = [ os.path.join(cwd, 'davinci%i.txt' % i) for i in range(n)]

In [15]:
import ipyparallel as ipp
rc = ipp.Client()
rc.ids

[0, 1, 2, 3]

In [16]:
view = rc.load_balanced_view()
eall = rc[:]
eall.push(dict(
    non_word=non_word,
    yield_words=yield_words,
    common_words=common_words,
    normalize_word=normalize_word,
))

<AsyncResult: _push>

### Exercise: parallel ngrams

Write a version of ngrams that runs in parallel,
rejoining the results into a single count dict.

In [18]:
# %load ../soln/ngrams.py
def ngrams_parallel(view, fnames, n=1):
    """Compute ngrams in parallel
    
    view - An IPython DirectView
    fnames - The filenames containing the split data.
    """

    ar = view.map_async(ngrams, fnames, [n] * len(fnames))
    counts = {}
    for engine_count in ar:
        for gram, count in engine_count.items():
            if gram not in counts:
                counts[gram] = 0
            counts[gram] += count
    return counts


In [19]:
print("Parallel ngrams")
%time pcounts = ngrams_parallel(view, fnames, 3)
print_common(pcounts, 10)

Parallel ngrams
CPU times: user 349 ms, sys: 28.9 ms, total: 378 ms
Wall time: 585 ms
  light and shade 98
     the same way 44
the luminous body 33
  between the eye 31
the space between 29
      pen and ink 29
leonardo da vinci 29
   the solar rays 27
   the right hand 27
space between the 27


### A bit more data

Download some Project Gutenberg samples from ntlk (avoid rate-limiting on PG itself)

In [20]:
gutenberg_samples = 'http://nltk.github.com/nltk_data/packages/corpora/gutenberg.zip'
if not os.path.isdir('gutenberg'):
    if not os.path.exists('gutenberg.zip'):
        urlretrieve(gutenberg_samples, 'gutenberg.zip')
    !unzip gutenberg.zip

import glob
gutenberg_files = glob.glob(os.path.abspath(os.path.join('gutenberg', '*.txt')))
# remove the bible, because it's too big relative to the rest
gutenberg_files.remove(os.path.abspath(os.path.join('gutenberg', 'bible-kjv.txt')))

In [21]:
ls gutenberg

README                   burgess-busterbrown.txt  milton-paradise.txt
austen-emma.txt          carroll-alice.txt        shakespeare-caesar.txt
austen-persuasion.txt    chesterton-ball.txt      shakespeare-hamlet.txt
austen-sense.txt         chesterton-brown.txt     shakespeare-macbeth.txt
bible-kjv.txt            chesterton-thursday.txt  whitman-leaves.txt
blake-poems.txt          edgeworth-parents.txt
bryant-stories.txt       melville-moby_dick.txt


In [22]:
print("Parallel ngrams across several books")
%time pcounts = ngrams_parallel(view, gutenberg_files, 3)
print()
print_common(pcounts, 10)
pcounts = ngrams_parallel(view, gutenberg_files, 4)
print()
print_common(pcounts, 10)


Parallel ngrams across several books
CPU times: user 1.81 s, sys: 88.3 ms, total: 1.9 s
Wall time: 2.85 s

     a great deal 175
       i dare say 107
farmer browns boy 88
  the sperm whale 86
    the same time 84
      i dont know 76
     two or three 74
    a few minutes 73
  the white whale 71
       mr and mrs 71

   at the same time 76
    a great deal of 66
 for the first time 48
     in a low voice 36
   i should like to 36
    out of the room 34
 of the sperm whale 31
much obliged to you 29
  i beg your pardon 28
 at the same moment 26
